# Training Data Collection

This notebook is used to create a training dataset to estimate snow depth from landsat data. The ground truth data is from SNOTEL stations.

Steps:
1. Setup: Selecting a station and setting some other constants
2. Use the lat/lon of the station to find landsat items, going back to 2004 (the furtherst back the SNOTEL API goes)
3. Use dates of those lansat items to query for snow depth from the SNOTEL API
4. Select band data from those landsat items at the lat/lon of the station (same lat/lon as snow depth measurement)
5. Save data to a parquet table

## Imports


In [1]:
import folium
import gc
from IPython.display import HTML
import fsspec
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from pystac_client import Client
from typing import Dict, Any
from concurrent.futures import ThreadPoolExecutor, as_completed

import data_classes
import plotting



/Users/aimeebarciauskas/github/ski-project/ski-project/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. Set constants

In [2]:
# Get lat/lon of SNOTEL stations
snotel_stations = pd.read_json("snotel-wa-stations.json")
snotel_api = "https://wcc.sc.egov.usda.gov/awdbRestApi/services/v1/data"
station_triplet = "1085:WA:SNTL"
station_lat, station_lon = snotel_stations[snotel_stations.stationTriplet == station_triplet].latitude.values[0], snotel_stations[snotel_stations.stationTriplet == station_triplet].longitude.values[0]
start_date, end_date = '2004-01-01', '2025-06-01'
stac_api = "https://landsatlook.usgs.gov/stac-server"
collections =  ["landsat-c2ard-sr"] # "landsat-c2l3-fsca" will also be used for snow cover fraction
fs = fsspec.filesystem("s3", anon=False, profile="barciaai", requester_pays=True)
parquet_file = 'snotel_data.parquet'


## 2. Search for landsat items

In [3]:
client = Client.open(stac_api)

In [4]:
station_buffer = 0.1
landsat_item_search = client.search(
    collections=collections,
    bbox=(station_lon-station_buffer, station_lat-station_buffer, station_lon+station_buffer, station_lat+station_buffer),
    datetime=(start_date, end_date)
)

landsat_item_search.matched()

1724

In [5]:
landsat_items = landsat_item_search.item_collection()

### Display HLS data and station

It is always helpful to visually inspect data to make sure we are on the right track!

In [6]:
first_landsat_item = landsat_items[0]


In [ ]:

# Create the map
m = folium.Map(
    location=[station_lat, station_lon],
    zoom_start=7,
    tiles='OpenStreetMap'
)

red = plotting.get_band_data(fs, first_landsat_item, 'red')
green = plotting.get_band_data(fs, first_landsat_item, 'green')
blue =plotting.get_band_data(fs, first_landsat_item, 'blue')
folium.raster_layers.ImageOverlay(
    name='True Color Image',
    image=f'data:image/png;base64,{plotting.rgb_image_str(red, green, blue)}',
    bounds=plotting.get_item_bounds(first_landsat_item),
    opacity=0.8,
    popup='HLS True Color Image'
).add_to(m)

folium.CircleMarker(
    location=[station_lat, station_lon],
    radius=5,
    color='green',
    fill=True,
    fillOpacity=0.7
).add_to(m)

# Display with custom size using HTML
html = m._repr_html_()
html_with_size = f'<div style="width: 500px; height: 500px;">{html}</div>'
HTML(html_with_size)

## 3. Create Parquet Schame and Table

Only create the table if it doesn't exist already

In [7]:
schema = pa.schema([
    pa.field('date', pa.string()),
    pa.field('snow_depth', pa.int64(), nullable=True),
    pa.field('red', pa.int64(), nullable=True),
    pa.field('green', pa.int64(), nullable=True),
    pa.field('blue', pa.int64(), nullable=True),
    pa.field('coastal', pa.int64(), nullable=True),
    pa.field('nir08', pa.int64(), nullable=True),
    pa.field('swir16', pa.int64(), nullable=True),
    pa.field('swir22', pa.int64(), nullable=True),
    pa.field('fsca', pa.int64(), nullable=True),
    pa.field('item_id', pa.string()),
    pa.field('station_triplet', pa.string()),  
])

In [8]:
create_new = True
if create_new == True:
    empty_table = schema.empty_table()
    pq.write_table(empty_table, parquet_file)

In [9]:
# Function for appending new data to parquet file
def append_items(file_path, new_data):
    new_table = pa.table(new_data, schema=schema)
    
    # Read existing data and append
    existing_table = pq.read_table(file_path)
    combined_table = pa.concat_tables([existing_table, new_table])
    
    # Write back to file
    pq.write_table(combined_table, file_path)

## 4. Load data from both landsat (band data) and SNOTEL API (ground truth) and store it in the table

1. Get SNOTEL snow depth data for the same date as each landsat scene
2. Read band data for the lat/lon of the SNOTEL data
3. Write data to the parquet table

In [10]:
batch_size = 10
start_idx = 0
end_idx = len(landsat_items)
max_workers = 10


In [31]:
import importlib
importlib.reload(data_classes)

<module 'data_classes' from '/Users/aimeebarciauskas/github/ski-project/data_classes.py'>

In [ ]:
def flatten_list(list_of_lists: list[list[Any]]) -> list[Any]:
    return [item for sublist in list_of_lists for item in sublist]

def process_item_parallel(args: tuple) -> Dict[str, Any]:
    """Process a single item in parallel"""
    fs, item, lat, lon, station_triplet = args
    extractor = data_classes.HLSDataExtractor(fs=fs, item=item)
    ground_truth = data_classes.SNOTELProvider(station_triplet=station_triplet)
    manager = data_classes.SatelliteDataManager(
        extractor=extractor,
        ground_truth_provider=ground_truth
    )    
    training_points = manager.extract_training_data([(lat, lon)])
    return manager.filter_valid_training_data(training_points)

# to test
# result = process_item_parallel((fs, first_landsat_item, station_lat, station_lon, station_triplet))
# append_items(parquet_file, data_classes.for_parquet_insert(flatten_list(result)))


In [ ]:
for i in range(start_idx, end_idx, batch_size):
    args_list = [(fs, item, station_lat, station_lon, station_triplet) for item in landsat_items[i:i+batch_size]]
    print(f"Processing items {i} to {i+batch_size} of {len(landsat_items)}")
    results = []

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all tasks
        future_to_item = {
            executor.submit(process_item_parallel, args): args[1] 
            for args in args_list
        }
        
        # Process completed tasks
        for future in as_completed(future_to_item):
            item = future_to_item[future]
            try:
                result = future.result()
                if result is not None:
                    results.append(result)
                print(f"Successfully processed item")
            except Exception as e:
                print(f"Error processing item {item.id}: {e}")

    append_items(parquet_file, data_classes.for_parquet_insert(flatten_list(results)))
    gc.collect()


Processing items 0 to 10 of 1724
Successfully processed item LC08_CU_004002_20250510_20250518_02_SR
Successfully processed item LC09_CU_004002_20250525_20250721_02_SR
Successfully processed item LC09_CU_004002_20250502_20250506_02_SR
Error processing item LC09_CU_004002_20250518_20250721_02_SR: 'NoneType' object has no attribute 'startswith'
Successfully processed item LC09_CU_004002_20250509_20250516_02_SR
Successfully processed item LC08_CU_004002_20250526_20250606_02_SR
Successfully processed item LC08_CU_004002_20250501_20250511_02_SR
Successfully processed item LC08_CU_004002_20250517_20250601_02_SR
Successfully processed item LC08_CU_004002_20250424_20250503_02_SR
Successfully processed item LC09_CU_004002_20250423_20250427_02_SR
Processing items 10 to 20 of 1724
Successfully processed item LC08_CU_004002_20250415_20250428_02_SR
Successfully processed item LC09_CU_004002_20250322_20250722_02_SR
Successfully processed item LC08_CU_004002_20250314_20250328_02_SR
Successfully proces

AttributeError: 'SnotelHlsData' object has no attribute 'coastal'

In [ ]:
df = pd.read_parquet(parquet_file)
df